# Reproducing NeurIPS2019 - Subspace Attack

As part of the NeurIPS2019 reproducibility challenge (project 2 of EPFL CS-433 2019) we chose to reproduce the paper [__Subspace Attack: Exploiting Promising Subspaces for Query-Efficient Black-box Attacks__](https://openreview.net/pdf?id=S1g-OVBl8r)

The algorithm is specified in: 

<img src="img/algo1.png" style="width:600px;"/>

We need to create the following functions:
- Load random reference model
- Loss function calculation
- Prior gradient calculation wrt dropout/layer ratio
- Attack

The pre-trained models are in (https://drive.google.com/file/d/1aXTmN2AyNLdZ8zOeyLzpVbRHZRZD0fW0/view).
The least demanding target model is the __GDAS__.


__Note!__ we start with 0 droupout ratio.